In [1]:
import os
from urllib.request import urlretrieve
import time
import zipfile
import shutil
import re
from tqdm import tqdm
import requests
import pandas as pd

In [2]:
year = 2016
quarter = 2
key_list = []
while True:
    year_str = str(year)[-2:]
    key_str = f'{year_str}-{quarter}T'
    key_list.append(key_str)
    quarter += 1
    if key_str == '24-1T':
        break
    if quarter == 5:
        quarter = 1
        year += 1
key_list

['16-2T',
 '16-3T',
 '16-4T',
 '17-1T',
 '17-2T',
 '17-3T',
 '17-4T',
 '18-1T',
 '18-2T',
 '18-3T',
 '18-4T',
 '19-1T',
 '19-2T',
 '19-3T',
 '19-4T',
 '20-1T',
 '20-2T',
 '20-3T',
 '20-4T',
 '21-1T',
 '21-2T',
 '21-3T',
 '21-4T',
 '22-1T',
 '22-2T',
 '22-3T',
 '22-4T',
 '23-1T',
 '23-2T',
 '23-3T',
 '23-4T',
 '24-1T']

In [3]:
value_list = []
for key in key_list:
    quarter = key[-2]
    year = '20' + key[:2]
    
    # Dealing with URLs for 2016
    if (year == '2016'):
        if quarter == '2':
            ordinal_str = 'do'            
        if quarter == '3':
            ordinal_str = 'er'
        if quarter == '4':
            ordinal_str = 'to'
        url_1 = 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/'
        url_2 = f'EPH_usu_{quarter}{ordinal_str}Trim_{year}_txt.zip'
        value_list.append(url_1 + url_2)
        continue

    # Dealing with the 2017-1T case
    if key == '17-1T':
        url_1 = 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/'
        url_2 = 'EPH_usu_1er_Trim_2017_txt.zip'
        value_list.append(url_1 + url_2)
        continue

    # The rest should work with this
    else:
        url_1 = 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/'
        url_2 = f'eph/EPH_usu_{quarter}_Trim_{year}_txt.zip'
        value_list.append(url_1 + url_2)

value_list

['https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2doTrim_2016_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3erTrim_2016_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4toTrim_2016_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_1er_Trim_2017_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2017_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2017_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4_Trim_2017_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_1_Trim_2018_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2018_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2018_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4_Trim_2018_txt.zip',
 'https://www.indec.gob.ar/ftp/cuadros/menusuperi

In [4]:
url_dict = dict(zip(key_list, value_list))
url_dict

{'16-2T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2doTrim_2016_txt.zip',
 '16-3T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3erTrim_2016_txt.zip',
 '16-4T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4toTrim_2016_txt.zip',
 '17-1T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_1er_Trim_2017_txt.zip',
 '17-2T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2017_txt.zip',
 '17-3T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2017_txt.zip',
 '17-4T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_4_Trim_2017_txt.zip',
 '18-1T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_1_Trim_2018_txt.zip',
 '18-2T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_2_Trim_2018_txt.zip',
 '18-3T': 'https://www.indec.gob.ar/ftp/cuadros/menusuperior/eph/EPH_usu_3_Trim_2018_txt.zip',
 '18-4T': 'https://www.indec.gob.ar/ftp/cuadr

In [5]:
# Create the "datos" folder if it doesn't exist
if 'datos' not in os.listdir():
    os.makedirs('datos')
    os.makedirs('datos/zip')

# Download the files
for key in tqdm(url_dict, desc='Downloading files'):
    url = url_dict[key]
    response = requests.get(url, stream=True)
    
    # If the website responds 
    if response.status_code == 200:
        with open(f'datos/zip/{key}.zip', 'wb') as f:
            f.write(response.content)

        # Try to avoid DDoS detection
        time.sleep(1)

    else:
        print(f'Failed to download {key}')

In [6]:
# Checking the files
error_files = 0
for file in os.listdir('datos/zip'):
    try:
        z = zipfile.ZipFile(f'datos/zip/{file}')
        z.testzip()
    except zipfile.BadZipFile:
        error_files += 1
        print(f'{file} is not a valid zip file')

if error_files == 0:
    print('It looks like all the zip files are valid')

It looks like all the zip files are valid


In [7]:
# Create the folders inside the "datos" folder
if 'individual' not in os.listdir('datos'):
    os.mkdir('datos/individual')
if 'hogar' not in os.listdir('datos'):
    os.mkdir('datos/hogar')

In [8]:
def extract_rename_and_move(key):
    """ This should extract the contents of the zip and extract the useful data 
    to the hogar and individual folders, which are inside the datos folder. 

    Args:
        key (str): a string that describes the year and quarter (trimestre)
        of the data. Should be something like 2018-3T.
    """
    # The filename for the .zip files
    zip_filename = key + '.zip'

    # Regex patterns to avoid problems with filenames
    hogar_re = r'[hH][oO][gG][aA][rR]'
    individual_re = r'[iI][nN][dD][iI][vV][iI][dD][uU][aA][lL]'

    # Creating the "extracted" folder
    if 'extracted' not in os.listdir('datos'):
        os.mkdir('datos/extracted')

    # Extracting the files
    z = zipfile.ZipFile(f'datos/zip/{zip_filename}')
    z.extractall('datos/extracted')

    # Searching the files
    for i in os.listdir('datos/extracted'):

        # If the data is inside a folder
        if os.path.isdir(f'datos/extracted/{i}'):
            folder_name = i
            for filename in os.listdir(f'datos/extracted/{folder_name}'):

                # Finding the "hogar" data
                if ((bool(re.search(hogar_re, filename))) and
                    filename.endswith('.txt')):
                    # Renaming the file
                    new_filename = f'{key}-hogar.txt'
                    os.rename(f'datos/extracted/{folder_name}/{filename}', 
                              f'datos/extracted/{folder_name}/{new_filename}')
                    # Moving the renamed file
                    shutil.move(f'datos/extracted/{folder_name}/{new_filename}',
                                f'datos/hogar/{new_filename}')

                # Finding the "individual" data
                if (((bool(re.search(individual_re, filename))) or
                     ('personas' in filename)) and
                    filename.endswith('.txt')):
                    # Renaming the file
                    new_filename = f'{key}-individual.txt'
                    os.rename(f'datos/extracted/{folder_name}/{filename}', 
                              f'datos/extracted/{folder_name}/{new_filename}')
                    # Moving the renamed file
                    shutil.move(f'datos/extracted/{folder_name}/{new_filename}',
                                f'datos/individual/{new_filename}')

        # If the data is not inside a folder
        else:

            # Finding the "hogar" data
            if (bool(re.search(hogar_re, i))) and i.endswith('.txt'):
                filename = i
                # Renaming the file
                new_filename = f'{key}-hogar.txt'
                os.rename(f'datos/extracted/{filename}', 
                          f'datos/extracted/{new_filename}')
                # Moving the renamed file
                shutil.move(f'datos/extracted/{new_filename}',
                            f'datos/hogar/{new_filename}')

            # Finding the "individual" data
            if (((bool(re.search(individual_re, i))) or ('personas' in i)) and
                i.endswith('.txt')):
                filename = i
                # Renaming the file
                new_filename = f'{key}-individual.txt'
                os.rename(f'datos/extracted/{filename}', 
                          f'datos/extracted/{new_filename}')
                # Moving the renamed file
                shutil.move(f'datos/extracted/{new_filename}',
                            f'datos/individual/{new_filename}')

    # Deleting the data inside the extracted folder
    shutil.rmtree('datos/extracted')

In [9]:
# Extracting, renaming and moving the data. Also deletes the zip folder
for key in key_list:
    extract_rename_and_move(key)
shutil.rmtree('datos/zip')

In [10]:
def read_data(key, hogar_or_individual):
    # Reading the data
    df = pd.read_csv(f'datos/{hogar_or_individual}/{key}-' +
                     f'{hogar_or_individual}.txt', sep=';', low_memory=False)
    
    # Converting int64 to int16 to save memory
    d = dict.fromkeys(df.select_dtypes('int').columns, 'int16')
    df = df.astype(d)
    return df

In [11]:
individual = read_data(key_list[0], 'individual')
for key in key_list[1:]:
    new_individual = read_data(key, 'individual')
    individual = pd.concat((individual, new_individual))
    del(new_individual)

individual

FileNotFoundError: [Errno 2] No such file or directory: 'datos/individual/22-3T-individual.txt'

In [12]:
individual

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
0,TQRMNOQTUHMMTQCDEIJAH00483381,2016,2,1,1,1,1,S,33,892,...,,10,"11866,67",09,,09,09,,09,1299
1,TQRMNOQTUHMMTQCDEIJAH00483381,2016,2,1,2,1,1,S,33,892,...,,10,"11866,67",09,,09,09,,09,1299
2,TQRMNOQTUHMMTQCDEIJAH00483381,2016,2,1,3,1,1,S,33,892,...,,10,"11866,67",09,,09,09,,09,1299
3,TQRMNOPSRHLOLRCDEIJAH00483068,2016,2,1,1,1,1,S,33,986,...,,10,"16333,33",10,,10,10,,10,1507
4,TQRMNORUVHLNMNCDEIJAH00484887,2016,2,1,4,1,1,S,33,1318,...,,09,6750,07,,06,07,,07,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58149,TQRMNOSURHJNMQCDEIJAH00505488,2016,4,1,6,1,1,S,33,716,...,,06,"2813,33",02,,02,02,,02,1613
58150,TQRMNOQQUHKOOLCDEIJAH00505489,2016,4,1,1,1,1,S,33,1752,...,,02,2250,02,,02,02,,02,2360
58151,TQRMNOQQUHKOOLCDEIJAH00505489,2016,4,1,2,1,1,S,33,1752,...,,02,2250,02,,02,02,,02,2360
58152,TQRMNOQQUHKOOLCDEIJAH00505489,2016,4,1,3,1,1,S,33,1752,...,,02,2250,02,,02,02,,02,2360


In [45]:
# Loading the data
individual = pd.read_csv(f'datos/individual/{key_list[0]}-individual.txt',
                         sep=';', low_memory=False)
# Converting int64 to int16 to save memory
d = dict.fromkeys(individual.select_dtypes('int').columns, 'int16')
individual = individual.astype(d)
# Converting CH05 to dates
individual['CH05'] = pd.to_datetime(individual['CH05'], format='%d/%m/%Y')

for key in key_list[1:]:
    more_individual
individual

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
0,TQRMNOQTUHMMTQCDEIJAH00483381,2016,2,1,1,1,1,S,33,892,...,,10,"11866,67",09,,09,09,,09,1299
1,TQRMNOQTUHMMTQCDEIJAH00483381,2016,2,1,2,1,1,S,33,892,...,,10,"11866,67",09,,09,09,,09,1299
2,TQRMNOQTUHMMTQCDEIJAH00483381,2016,2,1,3,1,1,S,33,892,...,,10,"11866,67",09,,09,09,,09,1299
3,TQRMNOPSRHLOLRCDEIJAH00483068,2016,2,1,1,1,1,S,33,986,...,,10,"16333,33",10,,10,10,,10,1507
4,TQRMNORUVHLNMNCDEIJAH00484887,2016,2,1,4,1,1,S,33,1318,...,,09,6750,07,,06,07,,07,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59806,TQRMNORPUHKLKMCDEOJAH00487302,2016,2,1,5,1,44,N,93,59,...,,,0,,,,,,,0
59807,TQRMNORPSHKLKMCDEOJAH00487300,2016,2,1,1,1,44,N,93,59,...,,,0,,,,,,,0
59808,TQRMNORPSHKLKMCDEOJAH00487300,2016,2,1,2,1,44,N,93,59,...,,,0,,,,,,,0
59809,TQRMNORPSHKLKMCDEOJAH00487300,2016,2,1,3,1,44,N,93,59,...,,,0,,,,,,,0


In [44]:
individual.select_dtypes('object')

,CODUSU,MAS_500,CH15_COD,CH16_COD,PP09A_ESP,PP09C_ESP,IDECOCUR,GDECOCUR,PDECOCUR,DECINDR,...,GDECIFR,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR
0,TQRMNOQWXHLMMLCDEFIAH00469008,S,NaN,NaN,NaN,NaN,00,00,,00,...,07,,07,8000,07,07,07,07,,06
1,TQRMNORPTHJMKTCDEFIAH00469111,S,PER,NaN,NaN,NaN,12,12,,12,...,,,,0,,,,,,
2,TQRMNORPTHJMKTCDEFIAH00469111,S,NaN,NaN,NaN,NaN,00,00,,00,...,,,,0,,,,,,
3,TQRMNORPTHJMKTCDEFIAH00469111,S,NaN,NaN,NaN,NaN,00,00,,00,...,,,,0,,,,,,
4,TQRMNORPTHJMKTCDEFIAH00469111,S,NaN,NaN,NaN,NaN,00,00,,00,...,,,,0,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59545,TQRMNOPSRHMOKMCDEOJAH00497870,N,NaN,NaN,NaN,NaN,00,,00,00,...,,,,0,,,,,,
59546,TQRMNOPSRHMOKMCDEOJAH00497870,N,NaN,NaN,NaN,NaN,00,,00,12,...,,,,0,,,,,,
59547,TQRMNOPSRHMOKMCDEOJAH00497870,N,NaN,NaN,NaN,NaN,12,,12,12,...,,,,0,,,,,,
59548,TQRMNOPSRHMOKMCDEOJAH00497870,N,NaN,NaN,NaN,NaN,00,,00,00,...,,,,0,,,,,,


In [23]:
individual.iloc[:, [102, 105, 134, 137, 143, 146]]

,PP09A_ESP,PP09C_ESP,IDECOCUR,PDECOCUR,IDECINDR,PDECINDR
0,NaN,NaN,,,,
1,NaN,NaN,,,,
2,NaN,NaN,,,,
3,NaN,NaN,,,,
4,NaN,NaN,,,,
...,...,...,...,...,...,...
59806,NaN,NaN,00,00,00,00
59807,NaN,NaN,12,12,12,12
59808,NaN,NaN,12,12,12,12
59809,NaN,NaN,00,00,00,00


In [26]:
individual['IDECOCUR'].dtypes

dtype('O')

In [11]:
%%time
drop_list = ['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'H15', 'AGLOMERADO', 'MAS_500', 
              'CH03', 'CH05', 'CH09', 'CH11', 'CH12', 'CH13', 'CH14', 'CH15',
              'CH15_COD', 'CH16', 'CH16_COD', 'IMPUTA', 'PP02C1', 'PP02C2',
              'PP02C3', 'PP02C4', 'PP02C5', 'PP02C6', 'PP02C7', 'PP02C8', 
              'PP03C', 'PP04B2', 'PP04B3_MES', 'PP04B3_ANO', 
              'PP04B3_DIA', 'PP04C99', 'PP05C_2', 'PP05C_3', 
              'PP05E', 'PP05F', 'PP05H', 'PP06A', 'PP06C', 'PP06D', 'PP06E', 
              'PP06H', 'PP07A', 'PP07D', 'PP07E', 'PP07F1', 'PP07F2', 
              'PP07F3', 'PP07F4', 'PP07F5', 'PP07G1', 'PP07G2', 'PP07G3', 
              'PP07G4', 'PP07G_59', 'PP07J', 'PP08D1', 'PP08D4', 'PP08F1', 
              'PP08F2', 'PP08J1', 'PP08J2', 'PP08J3', 'PP09A', 'PP09A_ESP',
              'PP09B', 'PP09C_ESP', 'DECOCUR', 'IDECOCUR', 'RDECOCUR', 
              'GDECOCUR', 'PDECOCUR', 'ADECOCUR', 'TOT_P12', 
              'DECINDR', 'IDECINDR', 'RDECINDR', 'GDECINDR', 'PDECINDR', 
              'ADECINDR', 'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M', 
              'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM', 'V21_M', 
              'T_VI', 'ITF', 'DECIFR', 'DECIFR', 'RDECIFR', 'GDECIFR', 'PDECIFR',
              'ADECIFR', 'IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 
              'PDECCFR', 'ADECCFR', 'PONDIH', 'PP09C_ESP', 'PP10A', 'PP10C', 
              'PP10D', 'PP10E', 'PP11A', 'PP11B_COD', 'PP11B1', 'PP11B2_MES', 
              'PP11B2_ANO', 'PP11B2_DIA', 'PP11C', 'PP11C99', 'PP11C99', 
              'PP11D_COD', 'PP11G_ANO', 'PP11G_MES', 'PP11G_DIA', 'PP11L', 
              'PP11L1', 'PP11M', 'PP11N', 'PP11O', 'PP11P', 'PP11Q', 'PP11R', 
              'PP11S', 'PP11T'
            ]

data_dict = {}
bad_files = []

for i in enumerate(files_list):
    try:
        data = pd.read_excel(i[1])
        data = data.drop(columns=drop_list)
        data = data.loc[data['REGION'] == 43]
        data_dict[f'eph{i[0]}'] = data
    except KeyError:
        bad_files.append(i[1])

NameError: name 'files_list' is not defined

In [12]:
bad_files

[]

In [13]:
eph = pd.concat([data_dict['eph1'], data_dict['eph2']], axis=0)
del(data_dict['eph1'])
del(data_dict['eph2'])

for i in enumerate(data_dict):
    eph =  pd.concat([eph, data_dict[i[1]]], axis=0)

KeyError: 'eph1'

In [ ]:
eph

,ANO4,TRIMESTRE,REGION,PONDERA,CH04,CH06,CH07,CH08,CH10,NIVEL_ED,...,PP07C,PP07H,PP07I,PP07K,PP09C,P21,PONDIIO,P47T,PONDII,IDECIFR
0,2017,1,43,861,2,18,1,4.0,2,2,...,NaN,NaN,NaN,NaN,NaN,0,861,1000.0,1088,
1,2017,1,43,861,2,13,5,4.0,1,3,...,NaN,NaN,NaN,NaN,NaN,0,861,0.0,861,
2,2017,1,43,861,1,15,5,4.0,1,3,...,NaN,NaN,NaN,NaN,NaN,0,861,0.0,861,
3,2017,1,43,861,2,11,5,4.0,1,3,...,NaN,NaN,NaN,NaN,NaN,0,861,0.0,861,
4,2017,1,43,861,1,7,5,4.0,1,1,...,NaN,NaN,NaN,NaN,NaN,0,861,0.0,861,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48540,2022,4,43,753,2,5,5,1.0,1,7,...,NaN,NaN,NaN,NaN,NaN,0,753,0.0,753,8.0
48541,2022,4,43,753,1,3,5,1.0,3,7,...,NaN,NaN,NaN,NaN,NaN,0,753,0.0,753,8.0
48542,2022,4,43,692,1,46,1,1.0,1,5,...,2.0,2.0,1.0,4.0,0.0,60000,933,60000.0,1089,9.0
48543,2022,4,43,692,2,44,1,1.0,2,6,...,2.0,1.0,0.0,1.0,0.0,160000,1033,160000.0,1092,9.0


In [ ]:
eph.loc[eph['ESTADO'] == 1, :].shape

(183461, 44)

In [ ]:
eph.loc[eph['P21'] != 0, :].shape

(179030, 44)

In [ ]:
eph.to_csv('EPH.csv', index=False)